In [ ]:
!pip install ЧегоНеХватает

In [1]:
import pandas as pd
import numpy as np
import lightgbm # сожрет все сырым и построит регрессионную модель, которая покажет важные фичи
                # чтобы дальше делать лабу только на них
                # если не можешь установить, то важные фичи в ячейке [8]
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
df = pd.read_csv('PIK_Property.csv', encoding='cp1251', index_col=0)
df.head(5)

,bulk_id,spalen,date1,start_square,value,price,mean_sq,mean_fl,plan_s,plan_m,...,До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет
id,,,,,,,,,,,,,,,,,,,,,
0,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-10-01,13109.9,1124.5,135633.913043,38.00,8,147,198,...,0.28,0.58,0.93,8,7640,57.698052,10.04,5.74,5.94,6.03
1,FF3814A9-9F7D-E711-8530-00505688958B,2,2017-10-01,19898.4,1412.3,128492.419825,58.01,9,139,120,...,0.28,0.58,0.93,8,7640,57.698052,10.04,5.74,5.94,6.03
2,FF3814A9-9F7D-E711-8530-00505688958B,0,2017-10-01,2100.0,345.0,158237.634409,22.58,10,78,15,...,0.28,0.58,0.93,8,7640,57.698052,10.04,5.74,5.94,6.03
3,FF3814A9-9F7D-E711-8530-00505688958B,3,2017-10-01,9614.9,769.4,123250.000000,78.81,8,50,0,...,0.28,0.58,0.93,8,7640,57.698052,10.04,5.74,5.94,6.03
4,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-11-01,11947.2,526.7,139420.634921,37.93,9,140,175,...,0.28,0.58,0.93,8,7640,58.926552,9.87,5.17,7.07,6.20


целевая переменная value

тут нет пропущенных значений, сделаем сами:

In [3]:
df.loc[0::5, ['mean_sq','bulk_id']] = np.nan # каждая пятая строка содержит пропущенные
df.head(5)

,bulk_id,spalen,date1,start_square,value,price,mean_sq,mean_fl,plan_s,plan_m,...,До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,1,2017-10-01,13109.9,1124.5,135633.913043,NaN,8,147,198,...,0.28,0.58,0.93,8,7640,57.698052,10.04,5.74,5.94,6.03
1,FF3814A9-9F7D-E711-8530-00505688958B,2,2017-10-01,19898.4,1412.3,128492.419825,58.01,9,139,120,...,0.28,0.58,0.93,8,7640,57.698052,10.04,5.74,5.94,6.03
2,FF3814A9-9F7D-E711-8530-00505688958B,0,2017-10-01,2100.0,345.0,158237.634409,22.58,10,78,15,...,0.28,0.58,0.93,8,7640,57.698052,10.04,5.74,5.94,6.03
3,FF3814A9-9F7D-E711-8530-00505688958B,3,2017-10-01,9614.9,769.4,123250.000000,78.81,8,50,0,...,0.28,0.58,0.93,8,7640,57.698052,10.04,5.74,5.94,6.03
4,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-11-01,11947.2,526.7,139420.634921,37.93,9,140,175,...,0.28,0.58,0.93,8,7640,58.926552,9.87,5.17,7.07,6.20


все колонки, которые не являются числами, указываем категориальными (чтобы lgbm их съел):

In [4]:
for column in df.select_dtypes(include = ['object']).columns.tolist():
    df[column] = df[column].astype('category')

In [5]:
lgbm_regressor = lightgbm.LGBMRegressor().fit(df.loc[:, df.columns != 'value'], df['value'])

In [6]:
list_of_importances = list(zip(df.loc[:, df.columns != 'value'].columns.tolist(), 
                               lgbm_regressor.feature_importances_))

In [7]:
sorted(list_of_importances, key= lambda x: x[1], reverse= True)

[('bulk_id', 366),
 ('start_square', 339),
 ('date1', 285),
 ('plan_l', 173),
 ('mean_sq', 168),
 ('plan_m', 155),
 ('vid_2', 131),
 ('price', 129),
 ('plan_s', 116),
 ('vid_1', 115),
 ('Площадь двора', 98),
 ('spalen', 84),
 ('До парка(км)', 64),
 ('До большой дороги на машине(км)', 58),
 ('vid_0', 53),
 ('mean_fl', 51),
 ('До метро пешком(км)', 46),
 ('month_cnt', 45),
 ('Площадь зеленой зоны в радиусе 500 м', 44),
 ('month', 43),
 ('Вклады свыше 3 лет', 40),
 ('Количество помещений', 39),
 ('Вклады от 1 года до 3 лет', 34),
 ('До удобной авторазвязки на машине(км)', 32),
 ('До парка пешком(км)', 28),
 ('До промки(км)', 25),
 ('Площадь пром. зоны в радиусе 500 м', 23),
 ('Cтавка по ипотеке', 22),
 ('Курс', 21),
 ('Детский сад', 19),
 ('Школа', 19),
 ('Вклады до 1 года', 18),
 ('Площадь земельного участка', 16),
 ('Машиномест', 15),
 ('До Садового(км)', 14),
 ('Станций метро от кольца', 14),
 ('До Кремля', 12),
 ('До ТТК(км)', 11),
 ('Класс объекта', 9),
 ('Двор без машин', 9),
 ('ФОК

In [8]:
important_features = [x[0] for x in sorted(list_of_importances, 
                                           key= lambda x: x[1], 
                                           reverse= True)[:19]]
important_features

['bulk_id',
 'start_square',
 'date1',
 'plan_l',
 'mean_sq',
 'plan_m',
 'vid_2',
 'price',
 'plan_s',
 'vid_1',
 'Площадь двора',
 'spalen',
 'До парка(км)',
 'До большой дороги на машине(км)',
 'vid_0',
 'mean_fl',
 'До метро пешком(км)',
 'month_cnt',
 'Площадь зеленой зоны в радиусе 500 м']

оставляем в датасете только важные фичи:

In [9]:
important_features.extend(['value'])
df = df[important_features]

### Лаба 3: пропуски, кодирование, масштабирование

заполнение пропусков:

In [10]:
for column in df.select_dtypes(include= ['int64', 'float64']).columns.tolist():
    df[column] = df[column].fillna(df[column].mean())
for column in df.select_dtypes(include= ['category']).columns.tolist():
    df[column] = df[column].fillna(df[column].describe(include= ['category'])['top'])

кодирование категориальных:

In [11]:
for column in df.select_dtypes(include= ['category']).columns.tolist():
    df = df.merge(df.groupby(column)['value'].mean().rename(str(column+'_value_mean')).to_frame(),
                  how= 'left',
                  left_on= column,
                  right_index= True)
    df.drop(column, axis=1, inplace=True)

масштабирование признаков:

In [12]:
df.loc[:, df.columns != 'value'] = df.loc[:, df.columns != 'value'].apply(lambda x: x/x.max(), 
                                                                          axis=0)

### Лаба 4: train-test, кросс-валидация, подбор параметров

трейн-тест:

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'value'], 
                                                    df['value'], 
                                                    test_size= 0.1, 
                                                    random_state= 42)

подбор лучшего параметра по кросс-валидации:

In [14]:
random_search = GridSearchCV(estimator= KNeighborsRegressor(),
                             param_grid= {'n_neighbors': [5,10,20,50]},
                             scoring= 'neg_mean_absolute_error', 
                             cv= 3)

random_search.fit(df.loc[:, df.columns != 'value'], df['value'])

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [5, 10, 20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [15]:
random_search.best_params_

{'n_neighbors': 50}

In [16]:
-random_search.best_score_ # минус потому что метрика 'negative_mean_absolute_error'

202.2916783427088